<a href="https://colab.research.google.com/github/koushikpr/Machine-Learning-Prerequisites/blob/Classification/Classification_Using_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

# **Classification**

---



2. Classification: Unlike linear Regression that predicts the output based on the numerical values plots on an n dimensional graph, Classification is going to seperate all these values based on features or cahracterstics of the data. 

Predicting the Flower species 
This specific dataset seperates flowers into 3 different classes of species.
- Setosa
- Versicolor
- Virginica

The information about each flower is the following.
- sepal length
- sepal width
- petal length
- petal width

In [ ]:
Columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
Species = ['Setosa', 'Versicolor', 'Virginica']

Step 1: Loading the DataSet of Flowers

In [ ]:
#keras utilities allows us to read files and retrieve them from a cloud or our device
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
#These are Prestored in the drive u can copy the above lines directly

#reading only the required column data
train = pd.read_csv(train_path, names=Columns, header=0)
test = pd.read_csv(test_path, names=Columns, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

Check the data

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Step 3: Remove the Species as we wanna predict that now

In [ ]:
trainspecies = train.pop('Species')
testspecies = test.pop('Species')
trainspecies.head()

0    2
1    1
2    2
3    0
4    0
5    0
6    0
7    2
8    1
9    0
Name: Species, dtype: int64

Step 4: Converting our Data to Tensors Input 

In [ ]:
def inputf(features,labels,training =True,batchsize=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    ds = ds.shuffle(1000).repeat()
  return ds.batch(batch_size=batchsize)
  

Step 5: Creating our feature columns

In [ ]:
fc = []
for key in train.keys():
  fc.append(tf.feature_column.numeric_column(key=key))
print(fc)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


Step 6: Loading the estimator DNN (Deep Neural Network Classifier)

In [ ]:
#DNN classifier with 2 hidden layers(30 and 10 nodes repsectively) and 3 classes
cf = tf.estimator.DNNClassifier(
    feature_columns=fc,
    hidden_units=[30,10],
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpo3k78gnj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Step 7: Training the estimator

In [ ]:
cf.train(
    input_fn = lambda: inputf(train,trainspecies,training=True),
    steps=5000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpo3k78gnj/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.608706, step = 0
INFO:tensorflow:global_step/sec: 329.079
INFO:tensorflow:loss = 1.1722572, step = 100 (0.309 sec)
INFO:tensorflow:global_step/sec: 460.305
INFO:tensorflow:loss = 1.0776241, step = 200 (0.219 sec)
INFO:tensorflow:global_step/sec: 398.584
INFO:tensorflow:loss = 1.0557326, step = 300 (0.251 sec)
INFO:tensorflow:global_step/sec: 426.101
INFO:tensorflow:loss = 1.0201087, step = 400 (0.235 sec)
INFO:tensorflow:global_step/sec: 393.658
INFO:tensorflow:loss = 1.0059271, step = 500 (0.253 sec)
INFO:te

Step 8: Evaluate the Model

In [ ]:
result = cf.evaluate(input_fn=lambda:inputf(test,testspecies,training=False))
clear_output()
print('Accuracy is',result['accuracy']*100)

Accuracy is 86.66666746139526


Step 9: Now Lets Run our predictions 

In [ ]:
predictions  = list(cf.predict(input_fn=lambda:inputf(test,testspecies,training=False)))
i = 6
#printing the detailsof the species
print(test.loc[i])
#extraction location of the species
id = predictions[i]['class_ids'][0]
#probability of only the required species and extracted location
probability = predictions[i]['probabilities'][id]
print('Species is ',Species[id],'Probability:',probability)
print('original species id',testspecies[i])
print('predicted id',id)
if testspecies[i]==id:
  print("Prediction successful")
else:
  print("Oops")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo3k78gnj/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
SepalLength    5.5
SepalWidth     4.2
PetalLength    1.4
PetalWidth     0.2
Name: 6, dtype: float64
Species is  Setosa Probability: 0.8486025
original species id 0
predicted id 0
Prediction successful


Step 10: User Input Prediction of the Flower

In [ ]:
#reading user data
def userinput(datass,batchsize=256):
  return tf.data.Dataset.from_tensor_slices(dict(datass)).batch(batch_size=batchsize)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

predict={}
#reading user data
print("enter numerical for following")
for f in features:
  valid =True
  while valid:
    val = input(f + ":")
    if not val.isdigit():
      valid=False
  predict[f]=[float(val)]
    
  

prediction = cf.predict(input_fn=lambda:userinput(predict))
#Displaying data using the data's id
clear_output()
for x in prediction:
  id = x['class_ids'][0]
  probability = x['probabilities'][id]

species = Species[id]
print("Species is ",species," probability:",probability)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo3k78gnj/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Species is  Virginica  probability: 0.57741416
